# Project Introduction
To implement a SVM classification machine-learning model, l choose the dataset downloaded from https://www.kaggle.com/abcsds/pokemon, which describes Pokémon’s stats and types. 
Pokémon is a game created by Nintendo in 1996 running in GBA devices. As soon as it released, it conquered the world. During a very long time, people are crazy about this game. However, as Nintendo met a huge problem in developing itself, the influence of this game also decreased as long the advent of intelligent equipment era, people are apt to play games running in their smartphone with huge screen rather than in GBA or some other out-of-time game devices. But, recently, Nintendo Company decided to release this game in smartphone, and this is a huge shock for the people who played this game which we could say it as “This game accompanies their growth”.
The reason why I want to choose this one as my dataset I because I am also a big fan of “Pokémon Go” and my objective is to make use of this dataset to help evaluating how good a Pokémon is, which means which sort of type could decide a Pokémon to be a legendary level guy. I hope to use what I have learned to contribute to this memorable stuff.


## Data Specification

In [1]:
# import packages
import pandas as pd
import numpy as np
from numpy import argmax
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn import linear_model
import numpy.polynomial.polynomial as poly

In [2]:
# read pokemon data from dataset
pk = pd.read_csv('Pokemon.csv', na_values = ['', ' ', 'NaN', np.nan], index_col = 0)
pk = np.array(pk)[:,1:]

In [3]:
# Data preprocessing
# merge two type into one
type_merge = []
for pair in zip(pk[:,0], pk[:,1]):
    pair = set(pair)
    type_merge.append(pair)
values = np.array(type_merge)
# transform types into one hot coding
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
onehot_encoder = OneHotEncoder(sparse = False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
pk = np.delete(pk, [0,1], 1)
pk = np.hstack((onehot_encoded, pk))
# transform bool to int
lb = LabelBinarizer()
pk[:,-1] = np.transpose(lb.fit_transform(pk[:,-1].tolist()))

In [4]:
# Take the first 160 rows (20%) to test
Xts = pk[:160,:-1]
Xtr = pk[161:,:-1]
yts = pk[:160,-1]
ytr = pk[161:,-1]

In [5]:
svc = svm.SVC(probability = False, kernel = "rbf", C = 100, gamma = 1, verbose = False)
svc.fit(Xtr, ytr.astype(float))
yhat = svc.predict(Xts)
acc = np.mean(yhat == yts)
print('Accuracy = {0:f}'.format(acc))

Accuracy = 0.981250


In [6]:
# read combats data from dataset
cb = pd.read_csv('combats.csv', na_values = ['', ' ', 'NaN', np.nan])
pk = pd.read_csv('pokemon_new.csv', na_values = ['', ' ', 'NaN', np.nan])
pk = np.array(pk)
type_merge = []
for pair in zip(pk[:,2], pk[:,3]):
    pair = set(pair)
    type_merge.append(pair)
values = np.array(type_merge)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
onehot_encoder = OneHotEncoder(sparse = False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
onehot_len = onehot_encoded.shape[1]
pk = np.delete(pk, [1,2,3], 1)
pk = np.insert(pk, [1], onehot_encoded, axis = 1)
pk = np.delete(pk, -1, 1) # delete the last column which represent whether it is legendary

cb = np.array(cb)
Xtr_cb = cb[:cb.shape[0]*4//5]
Xts_cb = cb[cb.shape[0]*4//5+1:]
ytr = []
yts = []
for pair in Xtr_cb:
    if pair[2] == pair[0]: ytr.append(0)
    else: ytr.append(1)
for pair in Xts_cb:
    if pair[2] == pair[0]: yts.append(0)
    else: yts.append(1)
ytr = np.array(ytr)
yts = np.array(yts)

In [7]:
Xtr_new = np.zeros((Xtr_cb.shape[0], pk.shape[1]-1+onehot_len))
Xts_new = np.zeros((Xts_cb.shape[0], pk.shape[1]-1+onehot_len))
for i, pair in enumerate(Xtr_cb):
    Xtr_new[i] = np.concatenate((pk[pair[0]-1,1:onehot_len+1], pk[pair[1]-1,1:onehot_len+1], pk[pair[0]-1,onehot_len+1:]-pk[pair[1]-1,onehot_len+1:]))
for i, pair in enumerate(Xts_cb):
    Xts_new[i] = np.concatenate((pk[pair[0]-1,1:onehot_len+1], pk[pair[1]-1,1:onehot_len+1], pk[pair[0]-1,onehot_len+1:]-pk[pair[1]-1,onehot_len+1:]))
regr = linear_model.LinearRegression()
print(Xtr_new.shape)
print(ytr.shape)
regr.fit(Xtr_new, ytr)
yhat = regr.predict(Xts_new)
for i, num in enumerate(yhat):
    if num < 0.5: yhat[i] = 0
    else: yhat[i] = 1
acc = np.mean(yhat == yts)
print('Accuracy = {0:f}'.format(acc))
RSS = np.mean((yhat-yts)**2/(np.std(yts)**2))
print('Normalized RSS = {0:f}'.format(RSS))

(40000, 1049)
(40000,)
Accuracy = 0.884088
0.464721764628
